# Strompreise

In [1]:
import time
from typing import Optional, Tuple, List
import datetime as dt
import pandas as pd
import pytz
import requests
import streamlit as st
import plotly.graph_objects as go

In [2]:
resolution_choice = "quarterly"

In [14]:
# ---------------------------------------------------------
# SMARD Loader
# ---------------------------------------------------------
SERIES_ID = "4169"
REGION_CANDIDATES = ["DE-LU", "DE"]
SMARD_BASE = "https://www.smard.de/app"
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; Streamlit-SMARD/1.0)"}
_last_tried: List[str] = []

class SmardError(Exception):
    pass

def _safe_get_json(url: str, timeout: int = 30) -> dict:
    _last_tried.append(url)
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    if r.status_code != 200:
        raise SmardError(f"HTTP {r.status_code} für {url}")
    try:
        return r.json()
    except Exception:
        snippet = (r.text or "")[:160].replace("\n", " ")
        ctype = r.headers.get("Content-Type", "")
        raise SmardError(f"Kein JSON von {url}. Content-Type='{ctype}', Antwort: '{snippet}...'")

def _get_index(region: str, resolution: str) -> list[int]:
    url = f"{SMARD_BASE}/chart_data/{SERIES_ID}/{region}/index_{resolution}.json"
    data = _safe_get_json(url)
    ts = data.get("timestamps") or []
    if not ts:
        raise SmardError(f"Keine timestamps in index_{resolution}.json ({region})")
    return ts

def _try_load_series(region: str, resolution: str, ts: int) -> Optional[pd.DataFrame]:
    for path in ["table_data", "chart_data"]:
        url = f"{SMARD_BASE}/{path}/{SERIES_ID}/{region}/{SERIES_ID}_{region}_{resolution}_{ts}.json"
        try:
            data = _safe_get_json(url)
            series = data.get("series")
            if series:
                return pd.DataFrame(series, columns=["ts_ms", "eur_per_mwh"])
        except SmardError:
            continue
    return None

# @st.cache_data(ttl=900)
def load_smard_series(prefer_resolution: str = "quarterhour", max_backsteps: int = 12) -> Tuple[pd.DataFrame, str, str]:
    resolutions = [prefer_resolution] + ([r for r in ["hour"] if r != prefer_resolution])
    for region in REGION_CANDIDATES:
        for resolution in resolutions:
            try:
                idx = _get_index(region, resolution)
            except SmardError:
                continue
            for ts in reversed(idx[-(max_backsteps + 1):]):
                df = _try_load_series(region, resolution, ts)
                if df is not None and not df.empty:
                    return df, resolution, region
                time.sleep(0.15)
    raise SmardError("Keine gültige SMARD-Datei gefunden (region/auflösung/ts).")

In [29]:
df_raw, used_resolution, used_region = load_smard_series(prefer_resolution=resolution_choice, max_backsteps=12)

In [30]:
df_raw

,ts_ms,eur_per_mwh
0,1760306400000,93.45
1,1760310000000,90.26
2,1760313600000,90.83
3,1760317200000,89.12
4,1760320800000,93.45
...,...,...
163,1760893200000,NaN
164,1760896800000,NaN
165,1760900400000,NaN
166,1760904000000,NaN


In [26]:
# ---------------------------------------------------------
# Data preparation
# ---------------------------------------------------------
tz_berlin = pytz.timezone("Europe/Berlin")
df_raw["ts"] = pd.to_datetime(df_raw["ts_ms"], unit="ms", utc=True).dt.tz_convert("Europe/Berlin")
df_raw["ct_per_kwh"] = df_raw["eur_per_mwh"] * 0.1

# ---------------------------------------------------------
# Zeitfenster: von aktuellem Mittag (12:00) bis nächstes Mittag (12:00)
# ---------------------------------------------------------
tz_berlin = pytz.timezone("Europe/Berlin")
now = dt.datetime.now(tz=tz_berlin)
today = now.date()

# define nominal window 12:00→12:00
start_window = tz_berlin.localize(dt.datetime.combine(today, dt.time(12, 0)))
end_window = start_window + dt.timedelta(days=1)

df_raw["ts"] = pd.to_datetime(df_raw["ts_ms"], unit="ms", utc=True).dt.tz_convert("Europe/Berlin")
df_raw["ct_per_kwh"] = df_raw["eur_per_mwh"] * 0.1

# Filter: keep data that falls inside that nominal 24-h window
df = df_raw[(df_raw["ts"] >= start_window - dt.timedelta(hours=12)) &
            (df_raw["ts"] < end_window + dt.timedelta(hours=12))].copy()

# ensure we actually cover the full delivery range; extend end to +12 h after last timestamp if needed
if not df.empty:
    last_ts = df["ts"].max()
    if last_ts < end_window:
        end_window = last_ts + dt.timedelta(hours=12)

# if df.empty:
#     st.info("Für dieses Zeitfenster liegen noch keine Day-Ahead-Daten vor.")
#     st.stop()

# # ---------------------------------------------------------
# # Komponenten: Spot + Gebühren (inkl. MwSt)
# # ---------------------------------------------------------
# fees = st.session_state.fees
# df["spot_ct"] = df["ct_per_kwh"]

# fees_no_vat = (
#     fees["stromsteuer_ct"]
#     + fees["umlagen_ct"]
#     + fees["konzessionsabgabe_ct"]
#     + fees["netzentgelt_ct"]
# )
# df["fees_incl_vat_ct"] = (fees_no_vat + df["spot_ct"]) * (fees["mwst"] / 100.0) + fees_no_vat
# df["total_ct"] = df["spot_ct"] + df["fees_incl_vat_ct"]


In [27]:
df_raw

,ts_ms,eur_per_mwh,ts,ct_per_kwh
0,1760306400000,93.45,2025-10-13 00:00:00+02:00,9.345
1,1760310000000,90.26,2025-10-13 01:00:00+02:00,9.026
2,1760313600000,90.83,2025-10-13 02:00:00+02:00,9.083
3,1760317200000,89.12,2025-10-13 03:00:00+02:00,8.912
4,1760320800000,93.45,2025-10-13 04:00:00+02:00,9.345
...,...,...,...,...
163,1760893200000,NaN,2025-10-19 19:00:00+02:00,NaN
164,1760896800000,NaN,2025-10-19 20:00:00+02:00,NaN
165,1760900400000,NaN,2025-10-19 21:00:00+02:00,NaN
166,1760904000000,NaN,2025-10-19 22:00:00+02:00,NaN


In [28]:
df

,ts_ms,eur_per_mwh,ts,ct_per_kwh
72,1760565600000,88.16,2025-10-16 00:00:00+02:00,8.816
73,1760569200000,85.81,2025-10-16 01:00:00+02:00,8.581
74,1760572800000,88.00,2025-10-16 02:00:00+02:00,8.800
75,1760576400000,86.66,2025-10-16 03:00:00+02:00,8.666
76,1760580000000,88.34,2025-10-16 04:00:00+02:00,8.834
77,1760583600000,85.05,2025-10-16 05:00:00+02:00,8.505
78,1760587200000,96.19,2025-10-16 06:00:00+02:00,9.619
79,1760590800000,116.12,2025-10-16 07:00:00+02:00,11.612
80,1760594400000,132.77,2025-10-16 08:00:00+02:00,13.277
81,1760598000000,119.93,2025-10-16 09:00:00+02:00,11.993


## Verschiedene Quellen

In [ ]:
# %reload_ext autoreload
# %autoreload 2

In [10]:
import price_sources

In [4]:
from datetime import datetime, timedelta, timezone
import pandas as pd

from price_sources import (
    fetch_smard_day_ahead,
    fetch_entsoe_day_ahead,
    fetch_smartenergy,
    _fmt_utc,
    DE_LU_EIC,
)

## SMARD

In [20]:
# 1) SMARD (no key)
df_smard = fetch_smard_day_ahead(resolution="quarterhour")

In [19]:
df_smard

,ts,eur_per_mwh,ct_per_kwh,source,resolution
0,2025-10-13 00:00:00+02:00,93.45,9.345,SMARD,hour
1,2025-10-13 01:00:00+02:00,90.26,9.026,SMARD,hour
2,2025-10-13 02:00:00+02:00,90.83,9.083,SMARD,hour
3,2025-10-13 03:00:00+02:00,89.12,8.912,SMARD,hour
4,2025-10-13 04:00:00+02:00,93.45,9.345,SMARD,hour
...,...,...,...,...,...
91,2025-10-17 19:00:00+02:00,130.69,13.069,SMARD,hour
92,2025-10-17 20:00:00+02:00,116.99,11.699,SMARD,hour
93,2025-10-17 21:00:00+02:00,104.05,10.405,SMARD,hour
94,2025-10-17 22:00:00+02:00,102.18,10.218,SMARD,hour


## ENTSOE

In [5]:
import os
import tomllib as tomli # Use 'import tomllib' if you are on Python 3.11+

# --- Load ENTSO-E token from secrets.toml ---

entsoe_token = None
secrets_path = os.path.join(".streamlit", "secrets.toml")

try:
    with open(secrets_path, "rb") as f:
        secrets = tomli.load(f) # Use 'tomllib.load(f)' for Python 3.11+
        entsoe_token = secrets.get("entsoe_token")

    if entsoe_token:
        print("✅ ENTSO-E token loaded successfully.")
        # You can now use the 'entsoe_token' variable in your API calls
        # For example:
        # df = price_sources.fetch_entsoe_day_ahead(token=entsoe_token, ...)
    else:
        print("⚠️ 'entsoe_token' not found in the secrets file.")

except FileNotFoundError:
    print(f"❌ Error: Secrets file not found at '{secrets_path}'.")
    print("   Please ensure the path is correct relative to your notebook's location.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

✅ ENTSO-E token loaded successfully.


#### Check XML structure

In [26]:
import datetime as dt
import requests
from xml.etree import ElementTree as ET
ENTSOE_BASE = "https://web-api.tp.entsoe.eu/api"
DE_LU_EIC = "10Y1001A1001A82H"  # Bidding zone code for DE/LU (BZN)


def fetch_entsoe_day_ahead_test(
    token: str,
    start_utc: dt.datetime,
    end_utc: dt.datetime,
    eic_bzn: str = DE_LU_EIC,
) -> pd.DataFrame:
    """
    Fetch Day-Ahead prices via ENTSO-E REST (XML), returning:
      ts (Europe/Berlin), eur_per_mwh, ct_per_kwh, source="ENTSOE", resolution ("PT15M"/"PT60M").
    start_utc / end_utc must be tz-aware in any tz (converted to UTC for the query).
    """
    if start_utc.tzinfo is None or end_utc.tzinfo is None:
        raise ValueError("start_utc and end_utc must be timezone-aware")

    params = {
        "securityToken": token,
        "documentType": "A44",
        "in_Domain": eic_bzn,
        "out_Domain": eic_bzn,
        "periodStart": _fmt_utc(start_utc),
        "periodEnd": _fmt_utc(end_utc),
    }

    resp = requests.get(ENTSOE_BASE, params=params, timeout=45)
    if resp.status_code == 401:
        raise PermissionError("ENTSO-E: Unauthorized (check token)")
    resp.raise_for_status()

    return resp

In [27]:
# 2) ENTSO-E (needs token) – query yesterday→tomorrow to be safe; UTC times
now_utc = dt.datetime.now(timezone.utc)
yest_utc = now_utc - timedelta(days=1)
tom_utc  = now_utc + timedelta(days=2)
resp = fetch_entsoe_day_ahead_test(entsoe_token, yest_utc, tom_utc, eic_bzn=DE_LU_EIC)


[autoreload of price_sources failed: Traceback (most recent call last):
  File "c:\Users\ChristophPromberger\dev\strompreise-app\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 322, in check
    elif self.deduper_reloader.maybe_reload_module(m):
         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
  File "c:\Users\ChristophPromberger\dev\strompreise-app\.venv\Lib\site-packages\IPython\extensions\deduperreload\deduperreload.py", line 545, in maybe_reload_module
    new_source_code = f.read()
  File "C:\Python313\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 13165: character maps to <undefined>
]


ConnectTimeout: HTTPSConnectionPool(host='web-api.tp.entsoe.eu', port=443): Max retries exceeded with url: /api?securityToken=97721bb0-d6cd-4aa6-9bbc-36c9d95b205c&documentType=A44&in_Domain=10Y1001A1001A82H&out_Domain=10Y1001A1001A82H&periodStart=202510200917&periodEnd=202510230917 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000025C2FDF2C10>, 'Connection to web-api.tp.entsoe.eu timed out. (connect timeout=45)'))

In [24]:
# Parse XML
ns = {"ns": "urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3"}
try:
    root = ET.fromstring(resp.content)
except ET.ParseError as ex:
    raise RuntimeError(f"ENTSO-E: XML parse error: {ex}")

NameError: name 'resp' is not defined

In [8]:
rows = []
# Day-ahead doc can include multiple TimeSeries
for ts_node in root.findall(".//ns:TimeSeries", ns):
    rows.append(ts_node)

In [9]:
root

<Element '{urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3}Publication_MarketDocument' at 0x000001A5FF46CE50>

In [11]:
rows

[<Element '{urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3}TimeSeries' at 0x000001A5FF46EC00>,
 <Element '{urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3}TimeSeries' at 0x000001A5FF567970>,
 <Element '{urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3}TimeSeries' at 0x000001A5FF55D8F0>,
 <Element '{urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3}TimeSeries' at 0x000001A5FF557830>,
 <Element '{urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3}TimeSeries' at 0x000001A5FF53D7B0>]

In [32]:
import xml.dom.minidom as md

xml_pretty = md.parseString(resp.content).toprettyxml(indent="  ")
print(xml_pretty)  # show first 2000 chars

<?xml version="1.0" ?>
<Publication_MarketDocument xmlns="urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3">
  
    
  <mRID>9af8768d90704dd9842c65bebecff52f</mRID>
  
    
  <revisionNumber>1</revisionNumber>
  
    
  <type>A44</type>
  
    
  <sender_MarketParticipant.mRID codingScheme="A01">10X1001A1001A450</sender_MarketParticipant.mRID>
  
    
  <sender_MarketParticipant.marketRole.type>A32</sender_MarketParticipant.marketRole.type>
  
    
  <receiver_MarketParticipant.mRID codingScheme="A01">10X1001A1001A450</receiver_MarketParticipant.mRID>
  
    
  <receiver_MarketParticipant.marketRole.type>A33</receiver_MarketParticipant.marketRole.type>
  
    
  <createdDateTime>2025-10-20T14:32:13Z</createdDateTime>
  
    
  <period.timeInterval>
    
      
    <start>2025-10-18T22:00Z</start>
    
      
    <end>2025-10-20T22:00Z</end>
    
    
  </period.timeInterval>
  
      
  <TimeSeries>
    
        
    <mRID>1</mRID>
    
        
    <auction.type>A01</auction.ty

### Check ENTSOE data structure

In [11]:
import importlib
importlib.reload(price_sources)
from price_sources import fetch_entsoe_day_ahead  #re-import

In [22]:
now_utc = dt.datetime.now(timezone.utc)
yest_utc = now_utc - timedelta(days=5)
tom_utc  = now_utc + timedelta(days=5)
DE_LU_EIC = "10Y1001A1001A82H"  # Bidding zone code for DE/LU (BZN)

df_entsoe = fetch_entsoe_day_ahead(entsoe_token, yest_utc, tom_utc, eic_bzn=DE_LU_EIC)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))

In [23]:
df_entsoe

,ts,eur_per_mwh,ct_per_kwh,source,resolution,mrid
0,2025-10-16 00:00:00+02:00,90.22,9.022,ENTSOE,PT15M,1
1,2025-10-16 00:00:00+02:00,103.00,10.300,ENTSOE,PT15M,2
2,2025-10-16 00:15:00+02:00,91.63,9.163,ENTSOE,PT15M,1
3,2025-10-16 00:15:00+02:00,87.59,8.759,ENTSOE,PT15M,2
4,2025-10-16 00:30:00+02:00,88.93,8.893,ENTSOE,PT15M,1
...,...,...,...,...,...,...
1042,2025-10-21 22:45:00+02:00,67.76,6.776,ENTSOE,PT15M,11
1043,2025-10-21 23:00:00+02:00,79.80,7.980,ENTSOE,PT15M,11
1044,2025-10-21 23:15:00+02:00,71.45,7.145,ENTSOE,PT15M,11
1045,2025-10-21 23:30:00+02:00,71.30,7.130,ENTSOE,PT15M,11


In [18]:
if not df_entsoe.empty:
    # Group by 'mrid' and aggregate to find the min and max timestamp for each group
    mrid_summary = df_entsoe.groupby('mrid')['ts'].agg(
        first_timestamp='min',
        last_timestamp='max'
    )
    
    print("First and last timestamp for each mRID in df_entsoe:")
    print(mrid_summary)
else:
    print("The DataFrame 'df_entsoe' is empty or has not been loaded yet.")

First and last timestamp for each mRID in df_entsoe:
               first_timestamp            last_timestamp
mrid                                                    
1    2025-10-16 00:00:00+02:00 2025-10-16 23:45:00+02:00
10   2025-10-20 00:00:00+02:00 2025-10-20 23:45:00+02:00
11   2025-10-21 00:00:00+02:00 2025-10-21 23:45:00+02:00
2    2025-10-16 00:00:00+02:00 2025-10-16 23:45:00+02:00
3    2025-10-17 00:00:00+02:00 2025-10-17 23:45:00+02:00
4    2025-10-17 00:00:00+02:00 2025-10-17 23:45:00+02:00
5    2025-10-18 00:00:00+02:00 2025-10-18 23:45:00+02:00
6    2025-10-18 00:00:00+02:00 2025-10-18 23:45:00+02:00
7    2025-10-19 00:00:00+02:00 2025-10-19 23:45:00+02:00
8    2025-10-19 00:00:00+02:00 2025-10-19 23:45:00+02:00
9    2025-10-20 00:00:00+02:00 2025-10-20 23:45:00+02:00


In [25]:
# Plot the different time series ---
if not df_entsoe.empty:
    fig = go.Figure()

    # Define a color map to assign a unique color to each mRID
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
    unique_mrids = sorted(df_entsoe['mrid'].unique())
    color_map = {mrid: colors[i % len(colors)] for i, mrid in enumerate(unique_mrids)}

    # Group by the 'mrid' column and create a trace for each group
    for mrid, group_df in df_entsoe.groupby('mrid'):
        
        # To fix the "overshoot" issue with step charts, add a final point
        # to explicitly define the end of the last time interval.
        plot_df = group_df.copy()
        if not plot_df.empty:
            last_row = plot_df.iloc[-1:].copy()
            if len(plot_df) > 1:
                time_diff = plot_df['ts'].iloc[-1] - plot_df['ts'].iloc[-2]
            else:
                # Guess interval if only one point exists
                time_diff = pd.Timedelta(minutes=15)
            last_row['ts'] = last_row['ts'] + time_diff
            plot_df = pd.concat([plot_df, last_row], ignore_index=True)
        
        fig.add_trace(go.Scatter(
            x=plot_df['ts'],
            y=plot_df['ct_per_kwh'],
            mode='lines',
            line_shape='hv',  # Vertical-then-horizontal step chart
            line=dict(color=color_map.get(mrid)), # Assign a specific color
            name=f"mRID: {mrid}",  # Name the trace using the mRID
            hovertemplate=f"mRID {mrid}: " + "%{y:.2f} ct/kWh<extra></extra>"
        ))

    fig.update_layout(
        title="ENTSO-E Day-Ahead Prices by Time Series (mRID)",
        xaxis_title="Time (Europe/Berlin)",
        yaxis_title="Price (ct/kWh)",
        hovermode="x unified",
        legend_title="Time Series"
    )

    fig.show()
else:
    print("DataFrame is empty, cannot generate plot.")

## smart Energy

In [ ]:

# 3) smartENERGY (if/when you have their JSON endpoint)
# Example placeholders – update url/fields once you have the real endpoint
# url = "https://www.smartenergy.at/api/spot"
# df_se = fetch_smartenergy(url, ts_field="timestamp", price_field="marketprice_eur_mwh", tz="Europe/Vienna")

# Compare coverage recency
def summarize(df, name):
    if df.empty:
        return f"{name}: empty"
    return f"{name}: {df['ts'].min()} → {df['ts'].max()} ({len(df)} rows)"

print(summarize(df_smard, "SMARD"))
print(summarize(df_entsoe, "ENTSO-E"))
# print(summarize(df_se, "smartENERGY"))

In [23]:
import price_sources

In [24]:
def load_price_data(
    source: str,
    resolution: str,
    entsoe_token: Optional[str],
    entsoe_days_back: int,
    entsoe_days_forward: int,
) -> tuple[pd.DataFrame, dict]:
    if source == "SMARD":
        df = price_sources.fetch_smard_day_ahead(resolution=resolution)
        meta = {"region": "DE-LU", "raw_resolution": resolution, "source_id": "SMARD"}
        return df, meta
    if source == "ENTSOE":
        if not entsoe_token:
            raise PriceDataError("Für ENTSO-E wird ein API Token benötigt.")
        now_utc = dt.datetime.now(dt.timezone.utc)
        start_utc = (now_utc - dt.timedelta(days=entsoe_days_back)).replace(minute=0, second=0, microsecond=0)
        end_utc = (now_utc + dt.timedelta(days=entsoe_days_forward)).replace(minute=0, second=0, microsecond=0)
        df = price_sources.fetch_entsoe_day_ahead(
            token=entsoe_token,
            start_utc=start_utc,
            end_utc=end_utc,
        )
        meta = {"region": "DE-LU", "raw_resolution": None, "source_id": "ENTSOE"}
        return df, meta
    raise PriceDataError(f"Unbekannte Datenquelle: {source}")

In [25]:
class PriceDataError(Exception):
    """Raised when a selected data source cannot provide usable data."""

data_source_choice = "ENTSOE"
resolution_choice = "quarterhour"
entsoe_token = "97721bb0-d6cd-4aa6-9bbc-36c9d95b205c"

try:
    bdf_raw, data_meta = load_price_data(
        source=data_source_choice,
        resolution=resolution_choice,
        entsoe_token=st.session_state.entsoe_token,
        entsoe_days_back=st.session_state.entsoe_days_back,
        entsoe_days_forward=st.session_state.entsoe_days_forward,
    )
except PriceDataError as exc:
    st.error(f"⚠️ {exc}")
    st.stop()
except Exception as exc:
    st.error(f"⚠️ Unerwarteter Fehler beim Laden der Datenquelle: {exc}")
    st.stop()
#used_region = data_meta.get("region", "DE-LU")
# resolution_fallback = normalize_resolution_hint(
    # data_meta.get("raw_resolution"), default=resolution_choice
# )

2025-10-20 16:21:35.870 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-20 16:21:35.871 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-20 16:21:35.872 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-20 16:21:35.873 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-20 16:21:35.874 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [26]:
df_raw, data_meta = load_price_data(
        source=data_source_choice,
        resolution=resolution_choice,
        entsoe_token=entsoe_token,
        entsoe_days_back=2,
        entsoe_days_forward=1,
    )

In [27]:
df_raw

,ts,eur_per_mwh,ct_per_kwh,source,resolution
0,2025-10-18 00:00:00+02:00,97.59,9.759,ENTSOE,PT15M
1,2025-10-18 00:00:00+02:00,93.61,9.361,ENTSOE,PT15M
2,2025-10-18 00:15:00+02:00,92.60,9.260,ENTSOE,PT15M
3,2025-10-18 00:15:00+02:00,103.62,10.362,ENTSOE,PT15M
4,2025-10-18 00:30:00+02:00,96.77,9.677,ENTSOE,PT15M
...,...,...,...,...,...
659,2025-10-21 22:45:00+02:00,67.76,6.776,ENTSOE,PT15M
660,2025-10-21 23:00:00+02:00,79.80,7.980,ENTSOE,PT15M
661,2025-10-21 23:15:00+02:00,71.45,7.145,ENTSOE,PT15M
662,2025-10-21 23:30:00+02:00,71.30,7.130,ENTSOE,PT15M
